# 简单等权策略

In [1]:
import os, sys, argparse, logging
import pandas as pd

# 首先将framework的目录加进sys.path中，方便使用相对路径导入
framework_path = os.path.join(os.path.abspath(''), '../framework')
if framework_path not in sys.path:
    sys.path.append(framework_path)

In [2]:
# 设置logging格式和级别
logging.basicConfig(
    level=logging.DEBUG,
    format='%(filename)s-line%(lineno)d %(levelname)s: %(message)s'
    )

## 1. 等权算法

In [3]:
from alg.alg_base import AlgBase

# 等权算法，传入资产字典和总权重
class EqualWeightAlg(AlgBase):
    def __init__(self, name) -> None:
        super().__init__(name)

    def run(self, asset_dict, weight_sum):
        logging.debug('EqualWeightAlg: run')
        return {asset: weight_sum/len(asset_dict) for asset in asset_dict}

In [4]:
# 测试算法

# 创建一个dummy的数据集
from component.asset.asset import Asset
dummy_dict = {i: Asset(str(i)) for i in range(3)}

ewa = EqualWeightAlg('equal_weight')
print(ewa.run(dummy_dict, 0.6))

880960759.py-line9 DEBUG: EqualWeightAlg: run


init YamlSvc
init DateSvc
{0: 0.19999999999999998, 1: 0.19999999999999998, 2: 0.19999999999999998}


## 2. 创建单期策略

In [5]:
from import_func import getSvc
raw_data_svc = getSvc('LxwWinddbRawDataSvc')
date_svc = getSvc('DateSvc')
date_svc.setTradeDays(raw_data_svc.getTradeDays())

from strategy.strategy_base import StrategyBase

from component.position_manager.group_position_manager import GroupPositionManager
from component.position_manager.asset_position_manager import AssetPositionManager

class MyStrategy(StrategyBase):
    def __init__(self, name, args={}) -> None:
        super().__init__(name, args={})

    # 设置需要使用的算法
    def _initAlgDict(self):
        self._alg_dict['equal_weight'] = EqualWeightAlg('equal_weight')

    # 设置数据集
    def _initDataset(self):
        # 父类方法初始化数据集并添加cash资产
        super()._initDataset(init_position_manager=True)

        a_000001 = Asset('000001.SH')
        a_000001.setRawNavData(raw_data_svc.getNav('aindexeodprices', '000001.SH'))

        a_000003 = Asset('000003.SH')
        a_000003.setRawNavData(raw_data_svc.getNav('aindexeodprices', '000003.SH'))

        self.getDataset().addChildAsset(a_000001)
        self.getDataset().addChildAsset(a_000003)

        # 如果进行回测，需要为每个资产初始化一个仓位管理器
        a_000001.setPositionManager(AssetPositionManager())
        a_000003.setPositionManager(AssetPositionManager())
        

    def run(self, id_date):
        logging.debug('{}: MyStrategy: run'.format(id_date))
        # 设置数据集的id_date
        self.getDataset().setIdDateRecursively(id_date, 252)

        # 构造算法要求的输入并执行算法
        alg_input = self.getDataset().getAllAsset(ignore_cash=True, id_date=id_date)
        return self._alg_dict['equal_weight'].run(alg_input, 0.5), []


lxw_winddb_sql_raw_data_svc.py-line68 DEBUG: raw_data_svc.sqlQuery: SELECT TRADE_DAYS FROM asharecalendar WHERE S_INFO_EXCHMARKET='SSE' ORDER BY TRADE_DAYS


init YamlSvc
init LxwWinddbRawDataSvc
init MysqlDbConnectorSvc


date_svc.py-line31 INFO: DateSvc: set trade days


init ConstantSvc


In [6]:
from datetime import datetime

# 测试单期策略
my_strategy = MyStrategy('test')

my_strategy._dataset.print()
print(my_strategy.run(datetime(2020, 1, 5)))

strategy_base.py-line47 INFO: strategy test: init dataset
lxw_winddb_local_raw_data_svc.py-line51 DEBUG: loading aindexeodprices from local_db
4051104135.py-line39 DEBUG: 2020-01-05 00:00:00: MyStrategy: run


group: root
	asset: cash
	asset: 000001.SH
	asset: 000003.SH


880960759.py-line9 DEBUG: EqualWeightAlg: run


({'000001.SH': 0.25, '000003.SH': 0.25}, [])


# 3. 执行回测

In [7]:
from backtest_manager.backtest_manager import BackTestManager

args = {
    'date_range': [[2010, 1, 1], [2010, 12, 31]], 
    'frequency': 'monthly', 
    'cash': 1e4, 
    }

my_backtest_manager = BackTestManager(
    MyStrategy('test'), # 策略实例
    name = ('equal_weight_backtest'), 
    args = args, 
)

my_backtest_manager.backtest()

strategy_base.py-line47 INFO: strategy test: init dataset
backtest_manager.py-line24 INFO: init BackTestManager
backtest_manager.py-line59 INFO: setting date index
backtest_manager.py-line66 INFO: setting init cash
strategy_base.py-line61 INFO: test: init cash
backtest:   0%|          | 0/242 [00:00<?, ?days/s]backtest_manager.py-line134 DEBUG: backtest: 2010-01-04 00:00:00
backtest_manager.py-line138 DEBUG: 2010-01-04 00:00:00, run strategy
4051104135.py-line39 DEBUG: 2010-01-04 00:00:00: MyStrategy: run
880960759.py-line9 DEBUG: EqualWeightAlg: run
backtest_manager.py-line145 DEBUG: 2010-01-04 00:00:00, update after close
c:\Users\Dell\Documents\SynologyDrive\实习\国君研究所\工作\FOF_portfolio\examples\../framework\component\position_manager\group_position_manager.py:33: RuntimeWarning: divide by zero encountered in double_scalars
  self.nav *= new_position / self.position
backtest_manager.py-line149 DEBUG: 2010-01-04 00:00:00, convert orders
backtest:   0%|          | 0/242 [00:00<?, ?days/s

AttributeError: 'Group' object has no attribute 'getAllAssets'